This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'
import glob
from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20180221'
file_list = glob.glob('../../dmu32/dmu32_SGP/data/tiles/sgp_{}_cigale_*.fits'.format(SUFFIX))

In [3]:
#len(master_catalogue)

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
# good = {}
# for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
#     good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
#                                    ~master_catalogue['flag_{}'.format(band)])
#     good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
#                               master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
# combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
# print("Number of good sources: {}".format(np.sum(combined_good)))

# Main catalogue for CIGALE

In [7]:
# best_catalogue = master_catalogue[combined_good].copy()

In [8]:
# # Correction for galactic extinction
# best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [9]:
# # Convertion to CIGALE format
# best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [10]:
u_bands = ["omegacam_u"]
g_bands = ["omegacam_g", "decam_g",            "gpc1_g"]
r_bands = ["omegacam_r", "decam_r",            "gpc1_r"]
i_bands = ["omegacam_i", "decam_i",            "gpc1_i"]
z_bands = [              "decam_z", "vista_z", "gpc1_z"]
y_bands = [              "decam_y", "vista_y", "gpc1_y"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [11]:
# remove_unneeded_fluxes(g_bands)
# remove_unneeded_fluxes(u_bands)
# remove_unneeded_fluxes(r_bands)
# remove_unneeded_fluxes(i_bands)
# remove_unneeded_fluxes(z_bands)
# remove_unneeded_fluxes(y_bands)

In [12]:
#best_catalogue.write("data_tmp/CDFS-SWIRE_cigale_best_extcor_20180129.fits")

# Catalogue using spectroscopic redshift

In [13]:
for tile in file_list:
        
    file_num = tile.split('_')[4].split('.')[0]
    best_catalogue = Table.read(tile)
    optnir = ((best_catalogue['flag_optnir_det'] == 3) 
          | (best_catalogue['flag_optnir_det'] == 7))
    
    best_catalogue = best_catalogue[optnir]


    best_catalogue = best_catalogue[~np.isnan(best_catalogue["zspec"])]
    best_catalogue["zspec"].name = "redshift"

    print("Number of sources with z-spec in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)


    best_catalogue.write("data_tmp/tiles/SGP_cigale_optnir_extcor_zspec_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)

Number of sources with z-spec in file 265: 54
Number of sources with z-spec in file 54: 50
Number of sources with z-spec in file 182: 96
Number of sources with z-spec in file 120: 92
Number of sources with z-spec in file 111: 113
Number of sources with z-spec in file 254: 38
Number of sources with z-spec in file 65: 91
Number of sources with z-spec in file 173: 51
Number of sources with z-spec in file 294: 162
Number of sources with z-spec in file 138: 14
Number of sources with z-spec in file 236: 86
Number of sources with z-spec in file 36: 87
Number of sources with z-spec in file 207: 87
Number of sources with z-spec in file 109: 102
Number of sources with z-spec in file 94: 81
Number of sources with z-spec in file 142: 26
Number of sources with z-spec in file 9: 76
Number of sources with z-spec in file 115: 93
Number of sources with z-spec in file 88: 101
Number of sources with z-spec in file 61: 67
Number of sources with z-spec in file 250: 74
Number of sources with z-spec in file 

INFO:herschelhelp.external:For 1 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Number of sources with z-spec in file 4: 65
Number of sources with z-spec in file 135: 9
Number of sources with z-spec in file 197: 123
Number of sources with z-spec in file 270: 23
Number of sources with z-spec in file 41: 100
Number of sources with z-spec in file 157: 71
Number of sources with z-spec in file 81: 85
Number of sources with z-spec in file 68: 50
Number of sources with z-spec in file 259: 41
Number of sources with z-spec in file 23: 69
Number of sources with z-spec in file 212: 54
Number of sources with z-spec in file 12: 67
Number of sources with z-spec in file 223: 79
Number of sources with z-spec in file 59: 22
Number of sources with z-spec in file 268: 90
Number of sources with z-spec in file 281: 32
Number of sources with z-spec in file 166: 52
Number of sources with z-spec in file 183: 102
Number of sources with z-spec in file 264: 62
Number of sources with z-spec in file 55: 47
Number of sources with z-spec in file 121: 93
Number of sources with z-spec in file 110